<a href="https://colab.research.google.com/github/Vijayalakshmi-42/selling_prediction/blob/main/Untitled17.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
import pandas as pd
#load dataset
df=pd.read_csv('car data.csv')
df.head()

,Car_Name,company,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,maruti suzuki,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,maruti suzuki,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,maruti suzuki,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,maruti suzuki,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,maruti suzuki,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [6]:
#drop car_name colum
df.drop('Car_Name',axis=1,inplace=True)
df.head()

,company,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,maruti suzuki,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,maruti suzuki,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,maruti suzuki,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,maruti suzuki,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,maruti suzuki,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [7]:
x=df.drop('Selling_Price',axis=1)
y=df['Selling_Price']

In [8]:
#train test split
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

In [16]:
from sklearn.ensemble import RandomForestRegressor
import pandas as pd

# Identify categorical columns from x_train
categorical_cols = x_train.select_dtypes(include=['object']).columns

# Perform one-hot encoding on x_train and x_test separately
x_train_encoded = pd.get_dummies(x_train, columns=categorical_cols, drop_first=True)
x_test_encoded = pd.get_dummies(x_test, columns=categorical_cols, drop_first=True)

# Align columns - crucial for consistent train/test sets if categories differ
# Get all unique columns from both encoded dataframes
all_columns = list(set(x_train_encoded.columns) | set(x_test_encoded.columns))

# Reindex both dataframes to have all columns, filling missing with 0
x_train_encoded = x_train_encoded.reindex(columns=all_columns, fill_value=0)
x_test_encoded = x_test_encoded.reindex(columns=all_columns, fill_value=0)

model=RandomForestRegressor(
n_estimators=200,
criterion='squared_error',
random_state=42
)
model.fit(x_train_encoded,y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [21]:
#revaluate model
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
y_pred=model.predict(x_test_encoded)
print('MSE:',mean_squared_error(y_test,y_pred))
print('MAE:',mean_absolute_error(y_test,y_pred))
print('R2:',r2_score(y_test,y_pred))

MSE: 0.8823794972540958
MAE: 0.6105745901639335
R2: 0.9616949378264364


In [32]:
#create prediction function
def predict_car_price(year,present_price,km_driven,fuel_type,seller_type,transmission,owner,company):
  # Create a DataFrame for the single input row
  input_df = pd.DataFrame([{
      'Year': year,
      'Present_Price': present_price,
      'Kms_Driven': km_driven,
      'Owner': owner,
      'Fuel_Type': fuel_type,
      'Seller_Type': seller_type,
      'Transmission': transmission,
      'company': company
  }])

  # Apply one-hot encoding to the input DataFrame
  input_encoded = pd.get_dummies(input_df, drop_first=True)

  # Ensure the columns of the input_encoded DataFrame match the columns of x_train_encoded
  # Any columns present in x_train_encoded but not in input_encoded will be added and filled with 0.
  # Any columns in input_encoded not in x_train_encoded will be dropped.
  missing_cols = set(x_train_encoded.columns) - set(input_encoded.columns)
  for c in missing_cols:
      input_encoded[c] = 0

  # Ensure the order of columns is the same as in x_train_encoded
  input_final = input_encoded[x_train_encoded.columns]

  prediction = model.predict(input_final)
  return prediction[0]

In [33]:
price=predict_car_price(
    year=2019,
    present_price=3.6,
    km_driven=20000,
    owner=1,
    company='Maruti',
    fuel_type='Petrol',
    seller_type='Individual',
    transmission='Manual'
)
print("pridicted selling price:",price)

pridicted selling price: 2.8210000000000046
